---
layout: post
title:  "Ch01.LangChain시작하기"
date:   2025-06-26 01:00:00 +0700
categories: [LLM]
---

<script type="text/x-mathjax-config">
MathJax.Hub.Config({tex2jax: {inlineMath: [['$','$'], ['\\(','\\)']]}});
</script>
<script type="text/javascript" src="https://cdn.mathjax.org/mathjax/latest/MathJax.js?config=TeX-MML-AM_CHTML">
</script>

### 참조 사이트.
해당 Post는 <a href="https://wikidocs.net/book/14314">LangChain 위키독스</a>에 나와있는 예제와 흐름을 파악하는 용도 입니다.  

### LangChain Expression Language(LECL)

기본 적인 Prompt + 모델 + 출력 파서에 대해 알아본다.

#### Import Library

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

#### Prompt 설정 방법

기본적으로 많이 사용하는 <code>PromptTemplate</code>는 아래와 같은 Argument로서 사용된다.

```python
PromptTemplate(
    input_variables: List[str],
    template: str,
    partial_variables: Optional[Dict[str, Any]] = None,
    validate_template: bool = True,
    template_format: str = "f-string"
)
```

- input_variables: 탬플릿에서 사용할 변수명들의 리스트
- template: Prompt 본문 문자열 (e.g., Hello {name})
- partial_variables: 일부 변수에 미리 값을 지정
- validate_template: 변수와 탬플릿이 일치하는지 검사
- template_format: f-string과 같은 Format 형식

실제 사용예시는 아래와 같다.

**기본 사용 방법**

In [2]:
prompt = PromptTemplate(
    input_variables=["product"], # 사용할 변수 명
    template="What is a good name for a company that makes {product}?" # 실제 Model에 넘길 Prompt
)

print(prompt.format(product="toys")) # 변수에 값을 넣어서 사용


What is a good name for a company that makes toys?


**partial_variables 예시**  

아래와 같이 partial_variables는 사전에 정의할 default variable이고, 추후 다른 값으로 변환 가능하다.

In [3]:
prompt = PromptTemplate(
    input_variables=["adjective", "noun"],
    template="The {adjective} {noun} is running fast.",
    partial_variables={"adjective": "quick"}
)

print(prompt.format(noun="fox"))
print(prompt.format(noun="fox", adjective="slow"))

The quick fox is running fast.
The slow fox is running fast.


#### Model Define

Local에서 Qwen3-4b model을 올려두고 사용한다. 아래와 같이 Local에서 띄운 vllm을 호출하여 사용할 수 있다.

In [4]:
model = ChatOpenAI(
    base_url="http://localhost:8000/v1",
    api_key="EMPTY",
    model="Qwen/Qwen3-4B"
)

#### StrOutputParser

LLM Model의 Output을 받아서 사용하기 위하여 사용된다. 주로 chain으로 연결된 LECL에서 많이 사용되며, 주로 사용되는 기능은 아래와 같다.

- <code>.invoke(output)</code>: 문자열 출력 결과를 반환 (일반적으로 내부적으로 사용됨)
- <code>.parse(text)</code>: 텍스트를 받아 가공 없이 문자열로 리턴
- <code>.parse_result(result)</code>: LLMResult 객체에서 첫 번째 텍스트 응답을 추출

#### LECL Example

실제 Prompt를 적용하고 LECL을 구성하여 Output을 확인한다.

In [5]:
# Template 선언
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 [FORMAT]에 영어 회화를 작성해 주세요.

상황:
{question}

FORMAT:
- 영어 회화:
- 한글 해석:
"""

# Prompt 생성
prompt = PromptTemplate.from_template(template)

# Local Server Vllm 선언
model = ChatOpenAI(
    base_url="http://localhost:8000/v1",
    api_key="EMPTY",
    model="Qwen/Qwen3-4B"
)

# 문자열 파서 선언
output_parser = StrOutputParser()

# LECL 선언
chain = prompt | model | output_parser

.invoke는 Model에 Prompt를 입력으로 넣어서 출력을 얻어내는 과정이다.

In [6]:
result = chain.invoke({"question": "레스토랑에서 주문하는 상황"})
print(result)

<think>
Okay, the user wants me to create an English conversation for a restaurant order scenario, with both English and Korean interpretations. Let me start by thinking about the typical interactions in a restaurant. The customer would start by greeting the server, then ask for the menu. The server would respond and maybe offer recommendations. The customer would then place an order, maybe ask about the menu items, and the server would confirm the order. Also, the customer might ask about the bill or special requests like drinks.

I need to make sure the conversation is natural and realistic. Let me list out the possible steps: greeting, ordering, asking about dishes, special requests, confirming the order, and ending with a thank you. Each part should have a simple, clear exchange. I should use common phrases that a customer would use in a restaurant. Also, the Korean translations need to be accurate, so I have to make sure the vocabulary matches the English words. For example, "What

.stream은 Model의 Output을 실시간으로 받아내는 과정이다. 아래 코드를 보면 아래와 같다.

In [ ]:
from langchain_core.messages import AIMessageChunk

'''
Model의 Output을 실시간으로 받아낸다. 
Generator로서 생성되며
각각의 값은 2가지가 될 수 있다.
    - AIMessageChunk: 여러 Token 묶음
    - token: 1개의 token
'''
response = chain.stream({"question": "미국에서 피자 주문"}) 

answer = ""
for token in response: # generator안의 값을 계속해서 받아온다.
    if isinstance(token, AIMessageChunk): # 여러 token이 들어오는 경우 for문을 돌며 각각의 token의 text를 확인한다.
        answer += token.content
        print(token.content, end="", flush=True)
    elif isinstance(token, str): # 1개의 token인 경우 바로 확인.
        answer += token
        print(token, end="", flush=True)

<think>
Okay, the user wants me to create an English conversation for ordering pizza in the US, with both English and Korean translations. Let me start by thinking about the typical steps in a pizza order. First, the customer would greet the cashier, maybe say hello. Then they'd ask for the menu or what's available. Next, they'd order the pizza, specifying the size, toppings, and any extra things like cheese or sauce. They might also ask about delivery or pickup. Then the cashier would confirm the order, and the customer thanks them.

I need to make sure the conversation is realistic and covers common scenarios. Let me check if there are any specific terms or phrases that are commonly used in US pizza places. For example, "large pizza," "pepperoni," "extra cheese," "hold the pepperoni," "delivery," "pickup," "total cost," "thank you."

Also, the Korean translation should be accurate. I need to make sure the translations are natural and not literal. For instance, "Would you like to orde

**Appendix: think token**: 해당 토큰은 현재 사용하는 vllm model (qwen3)의 특징이다. COT과정과 답변을 함께 출력하게 된다.

<hr>
참조: <a href="https://wikidocs.net/book/14314">LangChain 위키독스</a><br>
참조: <a href="https://github.com/wjddyd66/LLM/tree/main">원본 코드</a><br>

코드에 문제가 있거나 궁금한 점이 있으면 wjddyd66@naver.com으로  Mail을 남겨주세요.